In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Setup and Imports
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.layers import Bidirectional, GRU
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")

✓ All imports successful


In [3]:
def load_and_filter_fold(i):
    train_dir = f'/content/drive/MyDrive/split_data/fold{i}/train.csv'
    test_dir = f'/content/drive/MyDrive/split_data/fold{i}/test.csv'
    train_df = pd.read_csv(train_dir)
    test_df = pd.read_csv(test_dir)

    train_labels = list(train_df['room'].unique())
    test_labels = list(test_df['room'].unique())
    common_labels = list(set(train_labels) & set(test_labels))

    train_df = train_df[train_df['room'].isin(common_labels)].reset_index(drop=True)
    test_df = test_df[test_df['room'].isin(common_labels)].reset_index(drop=True)

    return train_df, test_df

# Load all 4 folds
train_df_1, test_df_1 = load_and_filter_fold(1)
train_df_2, test_df_2 = load_and_filter_fold(2)
train_df_3, test_df_3 = load_and_filter_fold(3)
train_df_4, test_df_4 = load_and_filter_fold(4)

print("✓ All folds loaded")

✓ All folds loaded


In [4]:
def set_seeds(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

def create_room_groups(df):
    df = df.sort_values('timestamp').reset_index(drop=True)
    df['room_group'] = (df['room'] != df['room'].shift()).cumsum()
    return df

def create_beacon_count_vectors(df):
    vectors = []
    for _, group in df.groupby('timestamp'):
        beacon_counts = group['mac address'].value_counts()
        total_readings = len(group)

        vector = [0.0] * 23
        for beacon_id, count in beacon_counts.items():
            if 1 <= beacon_id <= 23:
                vector[beacon_id - 1] = count / total_readings

        vectors.append({
            'timestamp': group['timestamp'].iloc[0],
            'room': group['room'].iloc[0],
            'room_group': group['room_group'].iloc[0],
            'beacon_vector': vector
        })

    return pd.DataFrame(vectors)

def create_sequences_from_groups(vector_df, min_length=3, max_length=50):
    sequences = []
    labels = []

    for (room, room_group), group in vector_df.groupby(['room', 'room_group']):
        group = group.sort_values('timestamp').reset_index(drop=False)
        seq_length = len(group)

        if seq_length < min_length:
            continue

        if seq_length > max_length:
            group = group.tail(max_length)
            seq_length = max_length

        sequence = [row['beacon_vector'] for _, row in group.iterrows()]
        sequences.append(sequence)
        labels.append(room)

    return sequences, labels

def build_gru_model(input_shape, num_classes):
    """
    Regular (unidirectional) GRU:
    - Even simpler than Bidirectional GRU
    - Might generalize even better to noisy data
    - Faster training
    """
    model = Sequential([
        Masking(mask_value=0.0, input_shape=input_shape),

        GRU(128, return_sequences=True),
        Dropout(0.3),

        GRU(64, return_sequences=False),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

print("✅ Helper functions defined (with regular GRU)")

✅ Helper functions defined (with regular GRU)


In [5]:
def run_pipeline_single_seed(train_df, test_df, seed, verbose=False):
    """Run pipeline for a single seed"""
    set_seeds(seed)

    # Preprocessing
    train_df = create_room_groups(train_df)
    test_df = create_room_groups(test_df)

    train_vectors = create_beacon_count_vectors(train_df)
    test_vectors = create_beacon_count_vectors(test_df)

    # Sequence creation
    X_train, y_train = create_sequences_from_groups(train_vectors)
    X_test, y_test = create_sequences_from_groups(test_vectors)

    # Encode labels
    label_encoder = LabelEncoder()
    label_encoder.fit(y_train + y_test)
    y_train_encoded = label_encoder.transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Pad sequences
    max_seq_length = 50
    X_train_padded = pad_sequences(X_train, maxlen=max_seq_length, dtype='float32', padding='post', value=0.0)
    X_test_padded = pad_sequences(X_test, maxlen=max_seq_length, dtype='float32', padding='post', value=0.0)

    # Calculate class weights
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train_encoded),
        y=y_train_encoded
    )
    class_weight_dict = dict(enumerate(class_weights))

    # Build and train model
    model = build_gru_model(
        input_shape=(max_seq_length, 23),
        num_classes=len(label_encoder.classes_)
    )

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=0)
    ]

    history = model.fit(
        X_train_padded, y_train_encoded,
        validation_data=(X_test_padded, y_test_encoded),
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        callbacks=callbacks,
        verbose=0
    )

    # Predictions (sequence level only - no frame propagation)
    y_pred_probs = model.predict(X_test_padded, verbose=0)
    y_pred_encoded = np.argmax(y_pred_probs, axis=1)
    y_pred = label_encoder.inverse_transform(y_pred_encoded)

    # Calculate macro F1
    macro_f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

    # Calculate per-class F1
    per_class_f1 = f1_score(y_test, y_pred, average=None, labels=label_encoder.classes_, zero_division=0)
    per_class_f1_dict = {label: f1 for label, f1 in zip(label_encoder.classes_, per_class_f1)}

    return {
        'seed': seed,
        'macro_f1': macro_f1,
        'per_class_f1': per_class_f1_dict
    }

print("✓ Pipeline function defined")

✓ Pipeline function defined


In [6]:
# Run 10 seeds for each of 4 folds
seeds = [42, 123, 456, 789, 2024, 3141, 5926, 8888, 1337, 9999]
folds = {
    1: (train_df_1, test_df_1),
    2: (train_df_2, test_df_2),
    3: (train_df_3, test_df_3),
    4: (train_df_4, test_df_4)
}

all_fold_results = {}

for fold_num, (train_df, test_df) in folds.items():
    print(f"\n{'='*80}")
    print(f"PROCESSING FOLD {fold_num}")
    print(f"{'='*80}\n")

    fold_results = []

    for seed in seeds:
        print(f"  Running seed {seed}...", end=" ")
        result = run_pipeline_single_seed(train_df, test_df, seed, verbose=False)
        fold_results.append(result)
        print(f"Macro F1: {result['macro_f1']:.4f}")

    all_fold_results[fold_num] = fold_results

    # Calculate fold statistics
    macro_f1_scores = [r['macro_f1'] for r in fold_results]
    print(f"\n  Fold {fold_num} Summary:")
    print(f"    Mean Macro F1: {np.mean(macro_f1_scores):.4f} ± {np.std(macro_f1_scores):.4f}")
    print(f"    Min: {np.min(macro_f1_scores):.4f}, Max: {np.max(macro_f1_scores):.4f}")

print("\n" + "="*80)
print("ALL FOLDS COMPLETED!")
print("="*80)


PROCESSING FOLD 1

  Running seed 42... Macro F1: 0.4806
  Running seed 123... Macro F1: 0.5221
  Running seed 456... 

Macro F1: 0.5358
  Running seed 789... Macro F1: 0.5672
  Running seed 2024... Macro F1: 0.4309
  Running seed 3141... Macro F1: 0.4997
  Running seed 5926... Macro F1: 0.5053
  Running seed 8888... Macro F1: 0.4389
  Running seed 1337... Macro F1: 0.5216
  Running seed 9999... Macro F1: 0.4467

  Fold 1 Summary:
    Mean Macro F1: 0.4949 ± 0.0427
    Min: 0.4309, Max: 0.5672

PROCESSING FOLD 2

  Running seed 42... Macro F1: 0.4226
  Running seed 123... Macro F1: 0.4384
  Running seed 456... Macro F1: 0.5043
  Running seed 789... Macro F1: 0.4660
  Running seed 2024... Macro F1: 0.4804
  Running seed 3141... Macro F1: 0.3136
  Running seed 5926... Macro F1: 0.4363
  Running seed 8888... Macro F1: 0.3096
  Running seed 1337... Macro F1: 0.4497
  Running seed 9999... Macro F1: 0.4292

  Fold 2 Summary:
    Mean Macro F1: 0.4250 ± 0.0614
    Min: 0.3096, Max: 0.5043

PROCESSING FOLD 3

  Running seed 42... Macro F1: 0.2870
  Running seed 123... Macro F1: 0.5707
  Running seed 456... Macr

In [7]:
# Save results to text file
with open('4fold_10seed_results.txt', 'w') as f:
    f.write("="*80 + "\n")
    f.write("4-FOLD CROSS-VALIDATION WITH 10 SEEDS PER FOLD\n")
    f.write("="*80 + "\n\n")

    # Overall summary
    all_macro_f1 = []
    for fold_num in [1, 2, 3, 4]:
        fold_scores = [r['macro_f1'] for r in all_fold_results[fold_num]]
        all_macro_f1.extend(fold_scores)

    f.write("OVERALL RESULTS (40 runs total):\n")
    f.write("-"*80 + "\n")
    f.write(f"Mean Macro F1: {np.mean(all_macro_f1):.4f} ± {np.std(all_macro_f1):.4f}\n")
    f.write(f"Min: {np.min(all_macro_f1):.4f}, Max: {np.max(all_macro_f1):.4f}\n\n")

    # Per-fold results
    for fold_num in [1, 2, 3, 4]:
        f.write(f"\n{'='*80}\n")
        f.write(f"FOLD {fold_num} RESULTS\n")
        f.write(f"{'='*80}\n\n")

        fold_results = all_fold_results[fold_num]
        macro_f1_scores = [r['macro_f1'] for r in fold_results]

        f.write(f"Macro F1 Scores (10 seeds):\n")
        f.write("-"*80 + "\n")
        for i, result in enumerate(fold_results):
            f.write(f"  Seed {result['seed']:5d}: {result['macro_f1']:.4f}\n")

        f.write(f"\nStatistics:\n")
        f.write(f"  Mean: {np.mean(macro_f1_scores):.4f} ± {np.std(macro_f1_scores):.4f}\n")
        f.write(f"  Min:  {np.min(macro_f1_scores):.4f}\n")
        f.write(f"  Max:  {np.max(macro_f1_scores):.4f}\n")

        # Per-class F1 (averaged across 10 seeds)
        f.write(f"\nPer-Class F1 Scores (averaged across 10 seeds):\n")
        f.write("-"*80 + "\n")

        # Collect all class names
        all_classes = set()
        for result in fold_results:
            all_classes.update(result['per_class_f1'].keys())

        # Average per-class F1 across seeds
        for class_name in sorted(all_classes):
            class_f1_scores = [r['per_class_f1'].get(class_name, 0) for r in fold_results]
            mean_f1 = np.mean(class_f1_scores)
            std_f1 = np.std(class_f1_scores)
            f.write(f"  {class_name:20s}: {mean_f1:.4f} ± {std_f1:.4f}\n")

print("✅ Results saved to 4fold_10seed_results.txt")

✅ Results saved to 4fold_10seed_results.txt


In [8]:
# Display summary
print("\n" + "="*80)
print("SUMMARY - 4 FOLDS × 10 SEEDS = 40 TOTAL RUNS")
print("="*80 + "\n")

for fold_num in [1, 2, 3, 4]:
    macro_f1_scores = [r['macro_f1'] for r in all_fold_results[fold_num]]
    print(f"Fold {fold_num}: {np.mean(macro_f1_scores):.4f} ± {np.std(macro_f1_scores):.4f}")

all_macro_f1 = []
for fold_num in [1, 2, 3, 4]:
    all_macro_f1.extend([r['macro_f1'] for r in all_fold_results[fold_num]])

print(f"\n{'='*80}")
print(f"Overall Mean: {np.mean(all_macro_f1):.4f} ± {np.std(all_macro_f1):.4f}")
print(f"{'='*80}")


SUMMARY - 4 FOLDS × 10 SEEDS = 40 TOTAL RUNS

Fold 1: 0.4949 ± 0.0427
Fold 2: 0.4250 ± 0.0614
Fold 3: 0.4816 ± 0.0843
Fold 4: 0.4778 ± 0.0771

Overall Mean: 0.4698 ± 0.0733
